In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
# from pyspark.sql.types import *
import ast

In [2]:
sc = SparkSession.builder \
            .master("local[2]") \
            .appName("book_impact_eda") \
            .getOrCreate()

sqlContext = SQLContext(sc)
df = sqlContext.read.load('/Users/harshita/Downloads/books_task.csv', 
                      format='com.databricks.spark.csv', 
                      header=True, 
                      inferSchema=True,
                      escape='"' 
                      )

23/12/31 00:27:15 WARN Utils: Your hostname, Harshitas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
23/12/31 00:27:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 00:27:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/harshita/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# df=df.drop("_c0")
df.show(4,truncate=False)

23/12/31 00:27:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+---+-------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
df.groupBy('publisher') \
.count() \
.sort("count", ascending=True) \
.show(truncate=False)

+-----------------------------+-----+
|publisher                    |count|
+-----------------------------+-----+
|L B I Law & Business Inst    |1    |
|Christian Writers Institute  |1    |
|X. J. Kennedy Poetry Prize   |1    |
|Velocity Pub                 |1    |
|Sarabande Books              |1    |
|Natural Law and Enlightenment|1    |
|Love Line Books              |1    |
|Rosarius Publishing House    |1    |
|Editorial Everest            |1    |
|OTexts                       |1    |
|Lawbook Exchange Limited     |1    |
|Muska & Lipman Pub           |1    |
|Gemma Halliday Publishing    |1    |
|Mathyz                       |1    |
|Emery Dalton Books           |1    |
|Rarebooksclub.com            |1    |
|Memoirs Unlimited            |1    |
|Univ Science Books           |1    |
|台灣五南圖書出版股份有限公司 |1    |
|Penticton, B.C. : R. Gardner |1    |
+-----------------------------+-----+
only showing top 20 rows



In [5]:
from pyspark.sql.functions import when, isnan, col, round

def find_row_missing_percentage(df):
    """
    This function takes a PySpark DataFrame as input and returns a dataframe after adding 
    columns that have missing values and the percentage of missing values in each row.

    :param df: PySpark DataFrame
    :return: PySpark DataFrame with percentage missing in each row
             List containing percentage of missing rows
    """
    # Count the number of columns
    total_columns = len(df.columns)

    # Find rows with missing values and count missing values in each row
    missing_values_df = df.select("*",
        sum(when(isnan(col(column_name)) | col(column_name).isNull(), 1).otherwise(0)
            for column_name in df.columns).alias("missing_count"))

    # Calculate the percentage of missing values for each row
    missing_percent_df = missing_values_df.withColumn("percent_missing", col("missing_count") / total_columns * 100)

    missing_percent_df = missing_percent_df.withColumn("percent_missing", round(missing_percent_df["percent_missing"], 0))

    # Calculate percentage of missing rows in dataframe 
    No_of_missRows = missing_percent_df.select('missing_count').where(missing_percent_df.missing_count>0).count()
    Percent_miss_rows = No_of_missRows/df.count()

    return missing_percent_df, Percent_miss_rows

In [6]:
df_misscount, Percent_miss_rows = find_row_missing_percentage(df)

# Print Percentage Missing Rows in the DataFrame
print("% Missing Rows:", "{:.2%}".format(Percent_miss_rows))

23/12/31 00:27:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


% Missing Rows: 10.32%


In [7]:
# Print Row value having with max % missing
your_max_value = df_misscount.agg({"percent_missing": "max"}).collect()[0][0]
print("Row value with max % missing:", your_max_value)

23/12/31 00:27:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


Row value with max % missing: 38.0


In [8]:
df_misscount.show(2)


+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+-------------+---------------+
|_c0|               Title|         description|         authors|           publisher|publishedDate|          categories|           Impact|missing_count|percent_missing|
+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+-------------+---------------+
|  0|Its Only Art If I...|                NULL|['Julie Strain']|Smithsonian Insti...|         1996|['Comics & Graphi...|784.3039243054303|            1|           13.0|
|  1|Dr. Seuss: Americ...|Philip Nel takes ...|  ['Philip Nel']|           A&C Black|   2005-01-01|['Biography & Aut...|825.4655354138016|            0|            0.0|
+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+-------------+---

23/12/31 00:27:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


In [9]:
# #Drop Columns with percent_missing > 60%
# missing_threshold = 60
# if Percent_miss_rows >= missing_threshold:
#     filtered_df = df_misscount.filter(df_misscount.percent_missing < missing_threshold)


In [10]:
df_misscount.count(), df.count()

(138724, 138724)

In [11]:
df.show(2)

+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+
|_c0|               Title|         description|         authors|           publisher|publishedDate|          categories|           Impact|
+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+
|  0|Its Only Art If I...|                NULL|['Julie Strain']|Smithsonian Insti...|         1996|['Comics & Graphi...|784.3039243054303|
|  1|Dr. Seuss: Americ...|Philip Nel takes ...|  ['Philip Nel']|           A&C Black|   2005-01-01|['Biography & Aut...|825.4655354138016|
+---+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+
only showing top 2 rows



23/12/31 00:27:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


In [12]:
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.types import IntegerType, StringType, NumericType

In [13]:
df.count()

138724

In [14]:
138724*0.10

13872.400000000001

In [15]:
df.groupBy('publisher') \
.count() \
.sort("count", ascending=True) \
.show(truncate=False)

+-----------------------------+-----+
|publisher                    |count|
+-----------------------------+-----+
|L B I Law & Business Inst    |1    |
|Christian Writers Institute  |1    |
|X. J. Kennedy Poetry Prize   |1    |
|Velocity Pub                 |1    |
|Sarabande Books              |1    |
|Natural Law and Enlightenment|1    |
|Love Line Books              |1    |
|Rosarius Publishing House    |1    |
|Editorial Everest            |1    |
|OTexts                       |1    |
|Lawbook Exchange Limited     |1    |
|Muska & Lipman Pub           |1    |
|Gemma Halliday Publishing    |1    |
|Mathyz                       |1    |
|Emery Dalton Books           |1    |
|Rarebooksclub.com            |1    |
|Memoirs Unlimited            |1    |
|Univ Science Books           |1    |
|台灣五南圖書出版股份有限公司 |1    |
|Penticton, B.C. : R. Gardner |1    |
+-----------------------------+-----+
only showing top 20 rows



In [16]:
import pyspark.sql.functions as fn
column = "publishedDate"
df = df.withColumn(f"{column}_year", fn.regexp_extract(column, r"\b(\d{4})\b", 1))
df = df.withColumn(f"{column}_month", fn.regexp_extract(column, r"(?<=-)(\d{2})", 0))


In [17]:
df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+
|_c0|               Title|         description|             authors|           publisher|publishedDate|          categories|           Impact|publishedDate_year|publishedDate_month|
+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+
|  0|Its Only Art If I...|                NULL|    ['Julie Strain']|Smithsonian Insti...|         1996|['Comics & Graphi...|784.3039243054303|              1996|                   |
|  1|Dr. Seuss: Americ...|Philip Nel takes ...|      ['Philip Nel']|           A&C Black|   2005-01-01|['Biography & Aut...|825.4655354138016|              2005|                 01|
|  2|Wonderful Worship...|This resource inc...|    ['David R. Ray']|             OUP USA| 

23/12/31 00:27:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


In [18]:
from pyspark.sql import functions as F
# Group by publisher_lower and count  
counts = df.groupBy("publisher").count()

# Join the counts back to the original DataFrame
df_publisher_map_count = df.join(counts, on="publisher", how="left") 

df_publisher_map_count = df_publisher_map_count.select("publisher", "_c0", F.col("count").alias("publisher_count"))


In [19]:
df_publisher_map_count.show()

23/12/31 00:27:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , publisher
 Schema: _c0, publisher
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+--------------------+---+---------------+
|           publisher|_c0|publisher_count|
+--------------------+---+---------------+
|Springer Science ...| 13|            885|
|Pickle Partners P...| 14|            234|
|Alaska Northwest ...| 19|              6|
|       Hill and Wang| 17|            313|
|           A&C Black|  1|           1355|
|              Bantam| 18|            511|
|Wm. B. Eerdmans P...|  5|           2563|
|     Tan Books & Pub|  8|           3635|
|            Sky Pony| 12|              1|
|Dorling Kindersle...| 23|            288|
|    Berg Pub Limited| 10|           1169|
|             OUP USA|  2|           1187|
|             OUP USA| 11|           1187|
|             OUP USA| 20|           1187|
|     Derrydale Press| 22|            355|
|  Plympton PressIntl|  9|              1|
| Seven Stories Press| 24|             69|
|           iUniverse|  3|           1057|
|Smithsonian Insti...|  0|           3216|
|Smithsonian Insti...| 21|           3216|
+----------

In [20]:
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Set the threshold for considering an author as 'unknown'
unknown_threshold = 5

# Replace the category with 'unknown' for less-known authors
df_result = df_publisher_map_count.withColumn(
    "publisher",
    when(col("publisher_count") < unknown_threshold, "unknown").otherwise(col("publisher"))
)

# Show the result
df_result.show()

23/12/31 00:28:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , publisher
 Schema: _c0, publisher
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+--------------------+---+---------------+
|           publisher|_c0|publisher_count|
+--------------------+---+---------------+
|Springer Science ...| 13|            885|
|Pickle Partners P...| 14|            234|
|Alaska Northwest ...| 19|              6|
|       Hill and Wang| 17|            313|
|           A&C Black|  1|           1355|
|              Bantam| 18|            511|
|Wm. B. Eerdmans P...|  5|           2563|
|     Tan Books & Pub|  8|           3635|
|             unknown| 12|              1|
|Dorling Kindersle...| 23|            288|
|    Berg Pub Limited| 10|           1169|
|             OUP USA|  2|           1187|
|             OUP USA| 11|           1187|
|             OUP USA| 20|           1187|
|     Derrydale Press| 22|            355|
|             unknown|  9|              1|
| Seven Stories Press| 24|             69|
|           iUniverse|  3|           1057|
|Smithsonian Insti...|  0|           3216|
|Smithsonian Insti...| 21|           3216|
+----------

In [21]:
# Set the threshold for considering an author as 'unknown'
unknown_threshold = 5

# Replace the category with 'unknown' for less-known authors
df_result = df_publisher_map_count.withColumn(
    "publisher",
    when(col("publisher_count") < unknown_threshold, "unknown").otherwise(col("publisher"))
)

# Show the result
df_result.show()

23/12/31 00:28:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , publisher
 Schema: _c0, publisher
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+--------------------+---+---------------+
|           publisher|_c0|publisher_count|
+--------------------+---+---------------+
|Springer Science ...| 13|            885|
|Pickle Partners P...| 14|            234|
|Alaska Northwest ...| 19|              6|
|       Hill and Wang| 17|            313|
|           A&C Black|  1|           1355|
|              Bantam| 18|            511|
|Wm. B. Eerdmans P...|  5|           2563|
|     Tan Books & Pub|  8|           3635|
|             unknown| 12|              1|
|Dorling Kindersle...| 23|            288|
|    Berg Pub Limited| 10|           1169|
|             OUP USA|  2|           1187|
|             OUP USA| 11|           1187|
|             OUP USA| 20|           1187|
|     Derrydale Press| 22|            355|
|             unknown|  9|              1|
| Seven Stories Press| 24|             69|
|           iUniverse|  3|           1057|
|Smithsonian Insti...|  0|           3216|
|Smithsonian Insti...| 21|           3216|
+----------

In [22]:
df_result \
.groupBy('publisher') \
.count().count()

2809

In [23]:
df_result.show()

23/12/31 00:28:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , publisher
 Schema: _c0, publisher
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+--------------------+---+---------------+
|           publisher|_c0|publisher_count|
+--------------------+---+---------------+
|Springer Science ...| 13|            885|
|Pickle Partners P...| 14|            234|
|Alaska Northwest ...| 19|              6|
|       Hill and Wang| 17|            313|
|           A&C Black|  1|           1355|
|              Bantam| 18|            511|
|Wm. B. Eerdmans P...|  5|           2563|
|     Tan Books & Pub|  8|           3635|
|             unknown| 12|              1|
|Dorling Kindersle...| 23|            288|
|    Berg Pub Limited| 10|           1169|
|             OUP USA|  2|           1187|
|             OUP USA| 11|           1187|
|             OUP USA| 20|           1187|
|     Derrydale Press| 22|            355|
|             unknown|  9|              1|
| Seven Stories Press| 24|             69|
|           iUniverse|  3|           1057|
|Smithsonian Insti...|  0|           3216|
|Smithsonian Insti...| 21|           3216|
+----------

In [24]:
df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+
|_c0|               Title|         description|             authors|           publisher|publishedDate|          categories|           Impact|publishedDate_year|publishedDate_month|
+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+
|  0|Its Only Art If I...|                NULL|    ['Julie Strain']|Smithsonian Insti...|         1996|['Comics & Graphi...|784.3039243054303|              1996|                   |
|  1|Dr. Seuss: Americ...|Philip Nel takes ...|      ['Philip Nel']|           A&C Black|   2005-01-01|['Biography & Aut...|825.4655354138016|              2005|                 01|
|  2|Wonderful Worship...|This resource inc...|    ['David R. Ray']|             OUP USA| 

23/12/31 00:28:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


In [25]:
df = df.join(df_result.select(["_c0", "publisher_count"]), on="_c0", how="left") 


In [26]:
# Top 2808 publishers and others category

In [27]:
#concat title and description
from pyspark.sql import functions as F

df = df.fillna("").withColumn(f"title_desc", F.concat(col("title"), col("description")))


In [28]:
df.columns

['_c0',
 'Title',
 'description',
 'authors',
 'publisher',
 'publishedDate',
 'categories',
 'Impact',
 'publishedDate_year',
 'publishedDate_month',
 'publisher_count',
 'title_desc']

In [29]:
drop_cols = ('_c0', 'Title', 'description','publishedDate', 'publisher_count')
df_final = df.drop(*drop_cols)

In [30]:
df.show(2)

23/12/31 00:28:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv
23/12/31 00:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , publisher
 Schema: _c0, publisher
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+---------------+--------------------+
|_c0|               Title|         description|             authors|           publisher|publishedDate|          categories|           Impact|publishedDate_year|publishedDate_month|publisher_count|          title_desc|
+---+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+------------------+-------------------+---------------+--------------------+
|467|The Devil in Sili...|This sweeping his...|['Stephen J. Pitti']|Princeton Univers...|   2018-06-05|         ['History']|841.7053210126119|              2018|                 06|            619|The Devil in Sili...|
|691|Beat Up a Cookie:...|Ellie Bernstein i...|    ['Denise Dietz']|  Wildside Press LLC|      2009-04|['Detective and m...|

In [31]:
# def identify_variable_types(df, unique_threshold=15000, id_vars=[]):
#     """
#     Identify variable types in a PySpark DataFrame.

#     :param df: The input PySpark DataFrame
#     :param unique_threshold: The maximum number of unique values for a discrete variable. Default is 10.
#     :id_vars: Unique keys like CustomerId, Names etc..
#     :return: A dictionary with variable names as keys and variable types as values.
#     """

#     discrete_columns = []
#     categorical_columns = []
#     continuous_columns = []
#     Other_columns = id_vars
#     df = df.drop(*Other_columns)

#     for column in df.columns:
#         dtype = df.schema[column].dataType

#         if isinstance(dtype, StringType):
#             unique_count = df.agg(countDistinct(col(column)).alias("unique_count")).collect()[0]["unique_count"]
#             if unique_count <= unique_threshold:
#                 categorical_columns.append(column)
#             else:
#                 Other_columns.append(column)

#         elif isinstance(dtype, IntegerType):
#             unique_count = df.agg(countDistinct(col(column)).alias("unique_count")).collect()[0]["unique_count"]
#             if unique_count <= unique_threshold:
#                 discrete_columns.append(column)
#             else:
#                 continuous_columns.append(column)

#         elif isinstance(dtype, NumericType):
#             continuous_columns.append(column)

#     return discrete_columns, categorical_columns, continuous_columns, Other_columns

In [32]:
# Identify variable types
# discrete_columns, categorical_columns, continuous_columns, Other_columns = identify_variable_types(df, id_vars=['RowNumber', 'CustomerId'])

# print("Discrete columns:", discrete_columns)
# print("Categorical columns:", categorical_columns)
# print("Continuous columns:", continuous_columns)
# print("Other columns:", Other_columns)


In [43]:
cat_columns = ['publisher', 'categories', 'Impact', 'publishedDate_year', 'publishedDate_month']

In [36]:
# cat_features = ['authors', 'publisher', 'categories']
text_features = ['title_desc']

In [37]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.functions import vector_to_array

In [44]:
def pipeline_fit_category_to_one_hot_encode(column_name, output_col_name):
    indexer = StringIndexer(inputCol=column_name, outputCol= output_col_name)
    return indexer

def pipeline_transform_category_to_one_hot_encode(column_name, output_col_name):

    encoder = OneHotEncoder(inputCols=[column_name], outputCols=[output_col_name], dropLast=True)
    return encoder

In [45]:
df_final.columns

['authors',
 'publisher',
 'categories',
 'Impact',
 'publishedDate_year',
 'publishedDate_month',
 'title_desc']

In [46]:
cat_columns

['publisher',
 'categories',
 'Impact',
 'publishedDate_year',
 'publishedDate_month']

In [41]:
selected_cols = ['Title', 'description', 'authors', 'publisher', 'publishedDate', 'categories', 'Impact']
df_final_v2 = df.select(*selected_cols)

#### Categorical Features

In [47]:
df_final_v2.show()

23/12/31 00:30:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, description, authors, publisher, publishedDate, categories, Impact
 Schema: _c0, Title, description, authors, publisher, publishedDate, categories, Impact
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv
23/12/31 00:30:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 
 Schema: _c0
Expected: _c0 but found: 
CSV file: file:///Users/harshita/Downloads/books_task.csv


+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+
|               Title|         description|             authors|           publisher|publishedDate|          categories|           Impact|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------+
|Its Only Art If I...|                    |    ['Julie Strain']|Smithsonian Insti...|         1996|['Comics & Graphi...|784.3039243054303|
|Dr. Seuss: Americ...|Philip Nel takes ...|      ['Philip Nel']|           A&C Black|   2005-01-01|['Biography & Aut...|825.4655354138016|
|Muslim Women's Ch...|Counters the West...|['Camillia Fawzi ...|    Berg Pub Limited|   1994-02-17|        ['Religion']|759.2711582939609|
|Dramatica for Scr...|Dramatica for Scr...|['Armando Salda A...|             OUP USA|      2005-07|       ['Reference']|847.0213715319953|
|Language:Social M...|Langu

In [ ]:
df_final=df_final_v2.drop("_c0")

In [ ]:
from pyspark.ml import Pipeline

stages = []
for col_name in cat_columns:
    stages.append(pipeline_fit_category_to_one_hot_encode(col_name, col_name+"_numeric"))
    # stages.append(pipeline_transform_category_to_one_hot_encode(col_name+"_numeric", col_name +'_onehot'))

pipeline = Pipeline(stages = stages)
df_transformed = pipeline.fit(df_final).transform(df_final)
